In [1]:
import pandas as pd
import numpy as np
import os
import sys
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from wtte.objectives.tensorflow import loglik_discrete, betapenalty
#output_notebook()

ImportError: No module named wtte.objectives.tensorflow

It is very telling to visualize the failures. What we can observe: we can observe the last time that the hardrive was seen in the data set. If the hard drive is no longer in the data set, it has failed, or has it just been removed from the data set? Our U matrix will be a mask the denotes censored drives, and most of our drives are censored. 

In [ ]:
pf = "../survival_data/survival_ST4000DM000.csv"
pf  = "../survival_data/survival_Hitachi_HDS723030ALA640.csv"
        df = pd.read_csv(pf, header=0)
        day = 24.0
        df_failures_only = df[df['failure'] ==1]
        ttf = df_failures_only['runtime_max']/day
        ttf_cen = df['runtime_max']/day
        et = df_failures_only['runtime_min']/day
        et_cen = df['runtime_min']/day

        u_train = np.abs(df_failures_only['failure']-1)
        u_train_cen = np.abs(df['failure']-1)

        sp1(ttf,ttf_cen, et, et_cen)
        sp2(ttf, ttf_cen, et, et_cen)
        p1(ttf, ttf_cen, et, et_cen, u_train, u_train_cen, plt_sorted = False)
        

In [16]:
DATA_FOLDER = "../survival_data/"
for data_file in os.listdir(DATA_FOLDER ):
    if data_file.split('.')[1] == 'csv':
        df = pd.read_csv(DATA_FOLDER + data_file, header=0)
        day = 24.0
        df_failures_only = df[df['failure'] ==1]
        ttf = df_failures_only['runtime_max']/day
        ttf_cen = df['runtime_max']/day
        et = df_failures_only['runtime_min']/day
        et_cen = df['runtime_min']/day

        u_train = np.abs(df_failures_only['failure']-1)
        u_train_cen = np.abs(df['failure']-1)

        if len(ttf) > 20:
            sp1(ttf,ttf_cen, et, et_cen, save_fig = data_file)
            sp2(ttf, ttf_cen, et, et_cen, save_fig = data_file)
            p1(ttf, ttf_cen, et, et_cen, u_train, u_train_cen, plt_sorted = False, save_fig = data_file)

survival_ST31500341AS.csv
125
survival_Hitachi_HDS5C4040ALE630.csv
58
survival_ST3160316AS.csv
survival_Hitachi_HDT725025VLA380.csv
survival_ST32000542AS.csv
survival_ST1000LM024_HN.csv
survival_ST2000DM001.csv
survival_ST2000VN000.csv
survival_ST250LT007.csv
survival_ST33000651AS.csv
survival_SAMSUNG_HD103UJ.csv
survival_Hitachi_HDS722020ALA330.csv
202
survival_HGST_HMS5C4040BLE640.csv
56
survival_ST4000DX002.csv
survival_Hitachi_HDS723020BLA642.csv
survival_TOSHIBA_MD04ABA500V.csv
survival_ST9250315AS.csv
survival_Hitachi_HDS5C3030BLE630.csv
survival_ST320005XXXX.csv
survival_ST2000DL001.csv
survival_TOSHIBA_MD04ABA400V.csv
survival_ST6000DM001.csv
survival_ST2000DL003.csv
survival_Hitachi_HDS724040ALE640.csv
survival_ST9320325AS.csv
survival_ST250LM004_HN.csv
survival_ST320LT007.csv
85
survival_TOSHIBA_MQ01ABF050.csv
survival_ST4000DX000.csv
36
survival_Hitachi_HDS723030ALA640.csv
63
survival_Hitachi_HDS5C3030ALA630.csv
110
survival_ST1500DM003.csv
survival_ST31500541AS.csv
274
surv

In [11]:
def sp1(ttf,ttf_cen, et, et_cen, save_fig = ""):
    sns.set(style="white", palette="muted", color_codes=True)
    sns.despine(top=True)
    binwidth = 30
    bw = 90
    bins_f = np.arange(min(ttf), max(ttf) + binwidth, binwidth)
    bins_c = np.arange(min(ttf_cen), max(ttf_cen) + binwidth, binwidth)

    f, axes = plt.subplots(2, 1, figsize=(14, 7), sharex=True)
    sns.distplot(ttf, color="red", ax=axes[0],  kde_kws={'bw':bw}, bins = bins_f)
    sns.distplot(et, color="grey", ax=axes[0],  kde_kws={'bw':bw}, bins = bins_f)
    sns.distplot(ttf_cen, color="blue", ax=axes[1], kde_kws={'bw':bw}, bins = bins_c)
    sns.distplot(et_cen, color="grey", ax=axes[1], kde_kws={'bw':bw}, bins = bins_c)
    axes[0].set_xlim([0, 1.02*ttf_cen.max()])
    axes[1].set_xlim([0, 1.02*ttf_cen.max()])
    axes[0].set_ylabel('Normalized Failure Count')
    axes[1].set_ylabel("Normalized Censor Count")
    axes[1].set_xlabel('Time')
    axes[0].set_xlabel('')
    plt.tight_layout()
    if save_fig != "":
        f.savefig(save_fig.split('.')[0] + '_a.png')
    else:
        plt.show()

In [13]:
def sp2(ttf, ttf_cen, et, et_cen, save_fig=""):
    binwidth = 30
    bw = 90
    
    f, axes = plt.subplots(2, 1, figsize=(14, 7), sharex=True)
    sns.despine(top=True)

    bins_f = np.arange(min(ttf), max(ttf) + binwidth, binwidth)
    bins_c = np.arange(min(ttf_cen), max(ttf_cen) + binwidth, binwidth)
    sns.distplot(ttf, color="red", ax=axes[0], kde_kws={'bw':bw,'cumulative':True}, 
                 hist_kws=dict(cumulative=True), bins = bins_f)
    sns.distplot(et, color="grey", ax=axes[0], kde_kws={'bw':bw,'cumulative':True}, 
                 hist_kws=dict(cumulative=True), bins = bins_f)

    sns.distplot(ttf_cen, color="blue", ax=axes[1], kde=False, 
                  hist_kws=dict(cumulative=True,normed=False), bins = bins_c)
    sns.distplot(et_cen, color="grey", ax=axes[1], kde=False,
                  hist_kws=dict(cumulative=True,normed=False), bins = bins_c)   
    sns.distplot(ttf, color="red", ax=axes[1], kde=False,
                  hist_kws=dict(cumulative=True,normed=False), bins = bins_c)   

    axes[0].set_xlim([0, 1.01*ttf_cen.max()])
    axes[1].set_xlim([0, 1.01*ttf_cen.max()])
    axes[0].set_ylabel('Normalized Cumulative Count')
    axes[1].set_ylabel("Cumulative Count")
    axes[1].set_xlabel('Time')
    axes[0].set_xlabel('')
    plt.tight_layout()
    if save_fig != "":
        f.savefig(save_fig.split('.')[0] + '_b.png')
    else:
        plt.show()

In [15]:
def p1(ttf, ttf_cen, et, et_cen, u_train, u_train_cen, plt_sorted = False, save_fig=""):
    sns.set_style("white")
    sns.despine(offset=10, trim=True);
    colorp =sns.xkcd_palette(["black", "red", "blue"])
    sns.set_palette(colorp)
    fig_n, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16, 9))
    
    if plt_sorted:
        ax1.scatter(sorted(ttf), range(len(ttf)), c=colorp[1], s=8, alpha=.9)
        
        ttf_cen_sorted = sorted(zip(ttf_cen, u_train_cen))
        ttf_cen_sorted, u_train_cen_sorted= [a[2] for a in ttf_cen_sorted],[a[1] for a in ttf_cen_sorted]
        u_colors_cen_sorted = [colorp[2] if c==1 else colorp[1] for c in u_train_cen_sorted]
        ax2.scatter(ttf_cen_sorted, range(len(ttf_cen_sorted)), c = u_colors_cen_sorted, s=8, alpha=.9)
    else:
        ax1.scatter(ttf, range(len(ttf)), c=colorp[1], s=8, alpha=.9)
        
        u_colors_cen = [colorp[2] if c==1 else colorp[1] for c in u_train_cen]
        ax2.scatter(ttf_cen, range(len(ttf_cen )), c = u_colors_cen, s=8, alpha=.9)
        ax2.scatter(et_cen, range(len(ttf_cen )), c = colorp[0], s=8, alpha=.9)
    
    ax1.set_ylabel('Drive')
    ax2.set_ylabel('Drive')
    ax1.set_title("Failures Only")
    ax2.set_title("Censored")
    ax2.set_xlabel('Time')
    ax1.set_xlabel('Time')
    ax1.set_xlim([0, 1.01*ttf_cen.max()])
    ax2.set_xlim([0, 1.01*ttf_cen.max()])
    if save_fig != "":
        fig_n.savefig(save_fig.split('.')[0] + '_c.png')
    else:
        plt.show()

Define fit with tensorflow and define a a penalty for beta.

In [ ]:
def betapenalty_alt(b, location = 10.0, growth=20.0, output_collection=(), name=None):
    """Returns a positive penalty term exploding when beta approaches location.

    Adding this term to the loss may prevent overfitting and numerical instability
    of large values of beta (overconfidence). Remember that loss = -loglik+penalty

    Args:
        b:  beta.  Positive nonzero `Tensor` of type `float32`, `float64`.
        output_collection: `tuple` of `string`s, name of the collection to
                          collect result of this op.
        name: `string`, name of the operation.

    Returns:
        A positive `Tensor` of same shape as `b` being a penalty term
    """
    with tf.name_scope(name, "weibull_betapenalty", [b]):
        scale = growth/location
        penalty = tf.exp(scale*(b-location))
        tf.add_to_collection(output_collection, penalty)

    return(penalty)


def tf_loglik_runner(loglik_fun, tte, tte_mask, n_steps = 2000):    
    y_ = tf.placeholder(tf.float32, shape=(None, 1))
    u_ = tf.placeholder(tf.float32, shape=(None, 1))
    
    # odd inital a for odd data
    a_guess = np.max(tte)
    a = tf.exp(tf.Variable(tf.random_normal([1], mean=a_guess, stddev=1.0), name='a_weight'))
    b = tf.exp(tf.Variable(tf.random_normal([1],mean= 1.0 , stddev=1.0), name='b_weight'))
     
    # added this betapenalty
    loglik = loglik_fun(a, b, y_,u_) 
    penalty = betapenalty_alt(b)
    loss = -tf.reduce_mean(loglik) + penalty
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.000001)
    
    train_step = optimizer.minimize(loss)
    sess = tf.Session()
    
    sess.run(tf.global_variables_initializer())

    for step in range(n_steps):
        loss_val,_,a_val,b_val= sess.run([loss,train_step, a,b],feed_dict={y_: tte, u_ : tte_mask})
        if step % 250 == 0:
            print("Step: %f, Loss value %s" % (step,loss_val))
            print ("Step: %f, A: %f B: %f" % (step,a_val, b_val))
    
    return a_val, b_val

discreteize data and shape it correctly for tensorflow

In [ ]:
ttf_cen = np.floor(ttf_cen)
ttf_cen_s = [[x] for x in ttf_cen]
u_train_cen_s =  [[x] for x in  u_train_cen]

ttf_f = np.floor(ttf)
ttf_f_s = [[x] for x in ttf_f]
u_train_f_s =  [[x] for x in  u_train]

In [ ]:
n_fit_steps = 2000


a_val_cen, b_val_cen = tf_loglik_runner(loglik_discrete, ttf_cen_s, u_train_cen_s, 
                                        n_steps = n_fit_steps)
print ("Cesnored case A: %f B: %f" % (a_val_cen[0], b_val_cen[0]))

a_val_f, b_val_f = tf_loglik_runner(loglik_discrete, ttf_f_s, u_train_f_s, 
                                        n_steps = n_fit_steps)
print ("Failures only case A: %f B: %f" % (a_val_f[0], b_val_f[0]))


Total fail. So lets just guess some Weibull params.

In [ ]:
b_val_f = 1.0
a_val_f = np.median(ttf)
W_f = np.sort(a_val_f * np.power(-np.log(np.linspace(1e-10,1.,len(ttf))),1.0/b_val_f))
W = np.sort(a_val_f * np.power(-np.log(np.linspace(1e-10,1.,len(ttf_cen_sorted))),1.0/b_val_f))

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16, 9))
ax1.scatter(sorted(ttf), range(len(ttf)), c=colorp[1], s=4, alpha=.2)
ax1.plot(W_f, range(len(ttf)), c="grey")
ax2.scatter(ttf_cen_sorted, range(len(ttf_cen_sorted)), c = u_colors_cen_sorted, s=4, alpha=.2)
ax2.plot(W, range(len(ttf_cen)), c="grey")
ax2.scatter(ttf_cen_sorted, range(len(ttf_cen_sorted)), c = u_colors_cen_sorted, s=4, alpha=.2)
ax1.set_ylabel('Drive')
ax2.set_ylabel('Drive')
ax1.set_title("Failures Only")
ax2.set_title("Censored")
ax2.set_xlabel('Time')
ax1.set_xlabel('Time')
ax1.set_xlim([0, 1.01*ttf_cen.max()])
ax2.set_xlim([0, 1.01*ttf_cen.max()])
plt.show()